Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
system('curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b76dd5a123e54a7eb42765499f9163a5&view=download_cytoscape_data" -o GNPS_output_graphML.zip')
system('unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip')

Download <i>in silico</i> structural annotation (Dereplicator, Varquest) data <br>
Replace Job IDs below with your Dereplicator or Varquest job IDs:

In [2]:
system('curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=4d971b8162644e869a68faa35f01b915&view=view_significant" -o Varquest_output.zip')
system('unzip -d Varquest_output/ Varquest_output.zip')

system('curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=c62d3283752f4f98b1720d0a6d1ee65b&view=view_significant" -o Derep_output.zip')
system('unzip -d Derep_output/ Derep_output.zip')

specify your NAP job ID:

In [3]:
nap_id = 'cb63770fe307410492468f62f9edb8f3'

load libraries

In [4]:
library('reticulate')
library('rjson')
library('igraph')


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [5]:
devtools::install_github("madeleineernst/RMolNetEnhancer", force = T)

✔  checking for file ‘/private/var/folders/fh/hzqxnlrs1gg7lmdpsfbknrqc0000gn/T/RtmpIOoCB2/remotes256a58eb167/madeleineernst-RMolNetEnhancer-6dde8c7/DESCRIPTION’
─  preparing ‘MolNetEnhancer’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘MolNetEnhancer_0.0.0.9000.tar.gz’
   


In [6]:
molnetenhancer <- import('pyMolNetEnhancer')
csv <- import('csv')
json <- import('json')

In [7]:
if ('clusterinfo_summary' %in% list.files('GNPS_output_graphML/') & 'DB_result' %in% list.files('GNPS_output_graphML/')){
    netfile <- paste0('GNPS_output_graphML/clusterinfo_summary/',
                        list.files('GNPS_output_graphML/clusterinfo_summary/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/DB_result/',
                        list.files('GNPS_output_graphML/DB_result/')[1])
} else if ('clusterinfosummarygroup_attributes_withIDs_withcomponentID' %in% list.files('GNPS_output_graphML/')){
    netfile <- paste0('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/',
                        list.files('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/result_specnets_DB/',
                        list.files('GNPS_output_graphML/result_specnets_DB/')[1])

} else {
    netfile <- paste0('GNPS_output_graphML/clusterinfosummary/',
                        list.files('GNPS_output_graphML/clusterinfosummary/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/result_specnets_DB/',
                        list.files('GNPS_output_graphML/result_specnets_DB/')[1])
}

In [8]:
nap <- read.csv(paste0("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task=",nap_id,
                       "&block=main&file=final_out/node_attributes_table.tsv"), sep = "\t", check.names = F)
gnpslib <- read.csv(gnpslibfile, sep='\t', check.names = F)
derep <- read.csv(paste0('Derep_output/',list.files('Derep_output/')[grep('DEREPLICATOR',list.files('Derep_output/'))]), sep = '\t', check.names = F)
varquest <- read.csv(paste0('Varquest_output/',list.files('Varquest_output/')[grep('DEREPLICATOR',list.files('Varquest_output/'))]), sep = '\t', check.names = F)

add all chemical structural information output as dataframe items in list

In [9]:
matches <- list(gnpslib, nap, derep, varquest)

In [10]:
out <- molnetenhancer$unique_smiles(matches)
smiles <- py_to_r(out$df)
colnames(smiles)[1] <- 'SMILES'
smiles <- smiles[-which(smiles[,1] == 'N/A'),,drop = FALSE]

In [11]:
write.csv(smiles,"SMILES.csv", quote = F, row.names = F)

convert SMILES to InchiKeys

In [12]:
system('/Applications/MarvinSuite/bin/molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt')

In [13]:
ikeys <- read.csv('InchiKeys.txt',sep='\t',header = F)
colnames(ikeys)[1] = "InChIKey"
out$df <- smiles
out$df$inchikey <- ikeys[,1]
inchi_dic <- molnetenhancer$make_inchidic(out)
write.csv(ikeys,"InchiKeys.txt", quote = F, row.names = F)

retrieve ClassyFire classifcations

In [14]:
molnetenhancer$get_classifications("InchiKeys.txt")

In [15]:
py <- import_builtins()
with(py$open("all_json.json") %as% tweetfile,{
    jsondic = json$loads(tweetfile$read())
})

df <- molnetenhancer$make_classy_table(jsondic)
colnames(df)[which(colnames(df) == 'class')] <- 'CF_class'
colnames(df)[which(colnames(df) == 'smiles')] <- 'SMILES'

net <- read.csv(netfile,  sep='\t', check.names = F)
final <- molnetenhancer$molfam_classes(net,df,dict(inchi_dic))

collists <- colnames(final)[which(sapply(final, class) == 'list')]
numcols <- c('CF_NrNodes',grep('score',colnames(final),value=TRUE))

for (i in 1:length(collists)){
    final[,which(colnames(final) == collists[i])] <- unlist(final[,which(colnames(final) == collists[i])])
}
for (i in 1:length(numcols)){
    final[,which(colnames(final) == numcols[i])] <- as.numeric(as.character(final[,which(colnames(final) == numcols[i])]))
}

In [16]:
head(final)

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
0,1,S1,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
1,6,S2,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
2,19,S3,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
3,22,S4,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
4,24,S5,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
5,25,30,2,Organic compounds,1,Benzenoids,0.75,Benzene and substituted derivatives,0.75,Benzyl alcohols,0.5,Benzyl alcohols,0.5,Aromatic homomonocyclic compounds,0.75


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [17]:
write.table(final,"ClassyFireResults_Network.txt", quote = F, row.names = F,  sep = '\t')

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [18]:
length(unique(ikeys$InChIKey)[-which(unique(ikeys$InChIKey)%in% df$inchikey)])

[1] 1729

which InChIKeys could not be classified?

In [19]:
unique(ikeys$InChIKey)[-which(unique(ikeys$InChIKey)%in% df$inchikey)]

[1] InChIKey=JNQOCTZBBDTBDQ-UHFFFAOYSA-N InChIKey=MYWHURBVYAKZKD-RRHRGVEJSA-N
   [3] InChIKey=XSJHUVBMPMRRPM-KKBITRJCSA-N InChIKey=AFYFLMDIXJQARK-MYACYKOGSA-N
   [5] InChIKey=VSVFHZOTKZOVAD-WOJBJXKFSA-O InChIKey=JNARUQNCSCUODB-HXVMWHSPSA-N
   [7] InChIKey=FLFGNMFWNBOBGE-UHFFFAOYSA-N InChIKey=PNNRCVLUZNDRRG-SGMTWIHVSA-N
   [9] InChIKey=ADKNUXINKGBJMY-LDLOPFEMSA-N InChIKey=IWIANZLCJVYEFX-UHFFFAOYSA-N
  [11] InChIKey=YIDWASJZCLLVPD-PJPVDIEGSA-N InChIKey=DHZUMZVZAZYQIW-USYZEHPZSA-N
  [13] InChIKey=PWAHARWXEAMNGK-QZNUWAOFSA-N InChIKey=QIJJGYHBXLEBJV-USYZEHPZSA-N
  [15] InChIKey=HPIOIUSFXDCSLD-UHFFFAOYSA-N InChIKey=DJHQMAXOQISYNY-MHJHVFNMSA-L
  [17] InChIKey=XWTHWSMCSBXZRJ-ZCMQOGSASA-N InChIKey=PDMUULPVBYQBBK-UHFFFAOYSA-N
  [19] InChIKey=MYXWLPFBAVVSPF-KXQOOQHDSA-N InChIKey=BPSLFSXCUJYFIR-UHFFFAOYSA-O
  [21] InChIKey=BINZSSJRRWYPEG-QSCHNALKSA-N InChIKey=XKJIHESQQBIMNO-SBBSCZBTSA-N
  [23] InChIKey=KDEGUFHTTWTTCT-VZUYHUTRSA-N InChIKey=PDIGSOAOQOXRDU-HHHXNRCGSA-N
  [25] InChIKey=GWBFHOZOTORMQU-SEBHOYJNSA-O InChIKey=MXMOXMNIWJUNPZ-KHTGXOHOSA-N
  [27] InChIKey=KKAZIKASZHKBKA-XJUOHMSHSA-O InChIKey=YGTUYXVZBPKODI-XSQGSSGQSA-N
  [29] InChIKey=LAFTYABRLNPQHQ-KXQOOQHDSA-N InChIKey=RPHRBJXFWYOOKX-WFIHMLKPSA-O
  [31] InChIKey=JZKVLKBVTSOCID-UHFFFAOYSA-N InChIKey=NSQZLRCRDQWFOY-UHFFFAOYSA-N
  [33] InChIKey=BETFSIOJEATXGY-UHFFFAOYSA-N InChIKey=ZDPLCZNXLVTAPJ-UHFFFAOYSA-N
  [35] InChIKey=YFDUWSBGVPBWKF-UHFFFAOYSA-N InChIKey=LEKBQPKGXGFBAN-UHFFFAOYSA-N
  [37] InChIKey=UDPBJAJIZZTTJS-ZDXQCDESSA-L InChIKey=WGHDGSZCVSIZQE-WWRGVMLBSA-N
  [39] InChIKey=WFNHXDBEYHHWAW-RPKWQDNBSA-N InChIKey=CJFRNMKINPNQHZ-NKIXGDIWSA-N
  [41] InChIKey=MTHBXTRAXHPTSN-UHFFFAOYSA-N InChIKey=LMWFNZUKABEGHS-LDLOPFEMSA-N
  [43] InChIKey=LPLUMFPBJGLPKS-UHFFFAOYSA-N InChIKey=XTUOVSKFZBWJCB-LEWJYISDSA-N
  [45] InChIKey=ROHDXJUFQVRDAV-UWVGGRQHSA-N InChIKey=OWSUNTVCUBCTDT-GTCPCGALSA-N
  [47] InChIKey=WLOYJVNVUHKSAT-SGQNLQFHSA-O InChIKey=UJZMLLMWTJZZJX-HUESYALOSA-N
  [49] InChIKey=ASSDFWWHWGKDER-OVQSNVQGSA-M InChIKey=KHWWHACQSMQRCE-DQEYMECFSA-O
  [51] InChIKey=BKCKPUXEOILIKW-UHFFFAOYSA-N InChIKey=VIOCEOKULYHXNJ-HUESYALOSA-N
  [53] InChIKey=PAPSGILQWHEGEX-ALHKGUTJSA-N InChIKey=LDZDUGYTLKWPEI-FTQMYMCWSA-N
  [55] InChIKey=VOBGWKUSIUTPDT-IQVVZXSMSA-O InChIKey=MSCCTZZBYHQMQJ-AZAGJHQNSA-N
  [57] InChIKey=VVPXASQNOLYBCR-TXCLPVNPSA-N InChIKey=OFJBEEKWIVBJST-KKKCCWTNSA-M
  [59] InChIKey=ANRCNDSZUNAAEU-RRHRGVEJSA-N InChIKey=MCLJQJFGZWVVNY-SWDWUIGYSA-P
  [61] InChIKey=AOXLESSRMGWHPT-MVCXZRGFSA-O InChIKey=SMALWXOJQUDDPA-ZAAXQIAVSA-N
  [63] InChIKey=PKUPANACYVQGRQ-FHCCRCCMSA-N InChIKey=VMNRUJGOLBSEPK-VIFPVBQESA-N
  [65] InChIKey=WEKNPHPVBWPNFG-WOJBJXKFSA-O InChIKey=JMZLLPMVWGNHBZ-JRFVFWCSSA-O
  [67] InChIKey=HMELXXITETZFSY-ALGHNRBBSA-N InChIKey=RVSJNDKZLRQIIG-RUZDIDTESA-N
  [69] InChIKey=FSHORJIJKLTTGL-ZZEZOPTASA-N InChIKey=ZGSNCXKJBYAIOL-UHFFFAOYSA-O
  [71] InChIKey=AZLHFAKIYLIXEW-UHFFFAOYSA-N InChIKey=AHEPKIJTHGCAOA-HUESYALOSA-N
  [73] InChIKey=VJTBJVHQCFQKLQ-XTNHTTTKSA-N InChIKey=ZUNZFMCDLSCUGC-TVHUAXGOSA-N
  [75] InChIKey=ZKWCOMLAOQTYIJ-UHFFFAOYSA-N InChIKey=FZLHEKWQJUWDCN-KDCQZWLBSA-O
  [77] InChIKey=UNAQSIZPRIMCTB-QXUJVPJQSA-N InChIKey=QYOHPKUFKYXMJH-RPIDORLJSA-N
  [79] InChIKey=YYUJOAOMEMVPHG-DFRNAFLYSA-N InChIKey=QCJAEZAXMNZVJH-NLOPCVKKSA-N
  [81] InChIKey=WLDZSRQOVLZRCO-FQEVSTJZSA-N InChIKey=XGQWRBZNQZVZFX-JMRSBHDGSA-O
  [83] InChIKey=MKQAFZMXXUQXAT-GIZNSFBFSA-N InChIKey=HFQNRECNIUNPMU-WBVITSLISA-N
  [85] InChIKey=IPHMCOQJRMKMMF-UHFFFAOYSA-N InChIKey=MNJXGNGAMWPKKE-ROIAFVFDSA-N
  [87] InChIKey=VIHZKKDXZBIOIK-HRVDDEAVSA-M InChIKey=DPALMNBCXDXMRW-GSDHBNRESA-O
  [89] InChIKey=DDJNYLNPKUVZPS-UQMNANQASA-N InChIKey=URYPPVMQQUERGK-UHFFFAOYSA-N
  [91] InChIKey=RWKODFJGLUVNKT-QPFCMRDRSA-N InChIKey=MRXKFVMGLVKXCN-RWLMNDHASA-N
  [93] InChIKey=IFEOHGBNFWCUOE-FNLZWABWSA-N InChIKey=VRJGMRRHBJEUSO-IVPBISSQSA-N
  [95] InChIKey=IDLUMGMMMVTISV-WBVITSLISA-N InChIKey=MKRBAPNEJMFMHU-UHFFFAOYSA-N
  [97] InChIKey=QZDZCTPKRWPFFS-UHFFFAOYSA-N InChIKey=CJBURLKHSGYKKJ-UHFFFAOYSA-N
  [99] InChIKey=XONAHZFJTGHACI-UHF

create graphML file

In [20]:
if (length(grep('FEATURE',list.files('GNPS_output_graphML/'))) != 0){
    graphMLfile <- paste0('GNPS_output_graphML/',list.files('GNPS_output_graphML/')[grep('FEATURE',list.files('GNPS_output_graphML/'))])
    graphML <- read_graph(graphMLfile, format = c("graphml"))
    graphML_classy <- MolNetEnhancer::make_classyfire_graphml(graphML, final)
    write_graph(graphML_classy, 'ClassyFireResults_Network.graphml', format = "graphml")
} else if (length(grep('METABOLOMICS',list.files('GNPS_output_graphML/'))) != 0) {
    graphMLfile <- paste0('GNPS_output_graphML/',list.files('GNPS_output_graphML/')[grep('METABOLOMICS',list.files('GNPS_output_graphML/'))])
    graphML <- read_graph(graphMLfile, format = c("graphml"))
    graphML_classy <- MolNetEnhancer::make_classyfire_graphml(graphML, final)
    write_graph(graphML_classy, 'ClassyFireResults_Network.graphml', format = "graphml")
} else {
    print('There is no graphML file for this GNPS molecular network job')
}